In [2]:
pip install pandas nltk pyodbc sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
# Download the VADER lexicon for sentiment analysis if not already present.
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\AL
[nltk_data]     WALEED\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [16]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

def fetch_data_from_sql():
    conn_str = (
        "Driver={SQL Server};"  
        "Server=DESKTOP-0SQSGS7\SQLEXPRESS;"  
        "Database=PortfolioProject_MarketingAnalytics;"  
        "Trusted_Connection=yes;" 
    )
    
    conn = pyodbc.connect(conn_str)
    
    
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM customer_reviews"
    
    df = pd.read_sql(query, conn)
    
    conn.close()
    
    return df

customer_reviews_df = fetch_data_from_sql()

sia = SentimentIntensityAnalyzer()

def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

def categorize_sentiment(score, rating):
    if score > 0.05:  
        if rating >= 4:
            return 'Positive'  
        elif rating == 3:
            return 'Mixed Positive'  
        else:
            return 'Mixed Negative'  
    elif score < -0.05:  
        if rating <= 2:
            return 'Negative'  
        elif rating == 3:
            return 'Mixed Negative'  
        else:
            return 'Mixed Positive'  
    else:  
        if rating >= 4:
            return 'Positive'  
        elif rating <= 2:
            return 'Negative'  
        else:
            return 'Neutral'  

def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  
    else:
        return '-1.0 to -0.5'  


customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

print(customer_reviews_df.head())

customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)

[nltk_data] Downloading package vader_lexicon to C:\Users\AL
[nltk_data]     WALEED\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
C:\Users\AL WALEED\AppData\Local\Temp\ipykernel_368\3799958936.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  
